# Bought By Many Interview Exercise

In [487]:
import os
import json
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime

%matplotlib inline

## 1. Process input files into CSVs

First, let's take a look at some policies to see what we're dealing with.

In [488]:
data_folder = 'data/policies'

def read_policy(filepath):
    
    with open(filepath) as file:
        text = file.read()
    
    return json.loads(text)

In [489]:
example = read_policy(f'{data_folder}/{os.listdir(data_folder)[0]}')

print(json.dumps(example, indent=2))

{
  "_id": "5d3dbfd80f17bd015b94cce4",
  "created_at": 1564327896418,
  "updated_at": 1578003371674,
  "account_ref": "BBM",
  "affiliate_code": "MSM",
  "cessation_date": 1595894400000,
  "data": {
    "insured_entities": [
      {
        "breed": "PAPILLON",
        "breed_friendly": "Papillon",
        "dob": 1546214400000,
        "gender": "MALE",
        "healthy": true,
        "microchipped": true,
        "pedigree": true,
        "pedigree_type": "PEDIGREE",
        "pet_address_postcode": "NN3",
        "pet_name": "George",
        "photo": "",
        "ref": "341692",
        "spayed_neutered": false,
        "species": "DOG",
        "uuid": "dcee9cc0-7987-40cb-a428-5bde29a1f057",
        "value": 1000
      }
    ],
    "payment": {
      "direct_debit_account_details": "gAAAAABdPb_ElU0tFUu144uNYZkLMiYwncHrQPK9ixRKauSkkGLR7TGgsqUg0MJ5YCOqxj03WlAXF-NiO3i5Nrt7aPRUI7afOY_cdyi1rFAS66IdO_PWhhE=",
      "direct_debit_account_name": "<removed>",
      "method": "IBSUITE_DIRECT

We see a lot of information, including details of:

* the policy holder
* the insured pets
* the insurance products included in the policy (allowances, pricing, rules etc.)
* payments
* event timelines (on risk, started, cancelled etc.)

We'll want the following in order to answer the questions in part 2:
* the country and date of each policy sold
* the species and breed of each pet
* the price of each product

We'll also want to relate policies to the pet(s) covered. Although unlikely, this has the potential to be a many-to-many relationship, since each policy can cover many pets, and it might be possible for each pet can be covered by multiple policies.

Similarly, we need to model the many-to-many relationship between policies and products. A policy's sales value is determined by the prices of the products included.

In [490]:
def select(dictionary, keys) -> dict:
    '''Filters a dictionary down to only the desired keys'''
    
    return {key: value for key, value in dictionary.items() if key in keys}

In [491]:
policies, pets, mapping = [], [], []

# read each policy file one by one and extract relevant data
for filename in os.listdir(data_folder):
    policy = read_policy(f'{data_folder}/{filename}')
    policy_id = policy.get('uuid')
    
    # calculate total price of products included in policy
    price = 0
    
    if 'products' in policy:
        
        for product in policy['products']:
            price += product['price']['annual']['amount']
    
    # extract relevant policy fields
    simple_policy = select(policy, {'uuid', 'account_ref', 'created_at'})
    simple_policy['price'] = price
    policies.append(simple_policy)
    
    # extract pets covered in policy
    if 'data' in policy and 'insured_entities' in policy['data']:
        covered_pets = policy['data']['insured_entities']

        for pet in covered_pets:
            pet_id = pet.get('uuid')
            # extract relevant pet fields
            pets.append(select(pet, {'uuid', 'species', 'breed'}))
            
            # if unique identifiers have been assigned as they should be...
            if policy_id and pet_id:
                # relate the policy to the pet in our many-to-many mapping table
                mapping.append({'policy_id': policy_id, 'pet_id': pet_id})

policies = pd.DataFrame(policies)
pets = pd.DataFrame(pets)
mapping = pd.DataFrame(mapping)

In [492]:
policies

,created_at,account_ref,uuid,price
0,1564327896418,BBM,e8f83f3e-6f12-4b2b-9ee0-06a285afa04b,33543
1,1561825294796,BBM,7163b853-2cdc-4a9a-bac3-59c9c8d7e54c,43565
2,1561050657706,BBM,f2a614fb-241f-42b8-a5c8-1f447fe46456,25221
3,1553939844000,BBM,6718260e-b401-4e9d-aa91-269ef30e01e1,49315
4,1569590695023,BBM,7cf0888b-f699-4b7d-aac4-f0277ca72852,59144
...,...,...,...,...
10521,1563696195609,BBM,db88159e-695e-4b1f-ba2d-6ac3d4a9e462,0
10522,1556959333710,BBM,886a3469-9a62-4646-bc19-41f59188e32d,36807
10523,1555184687260,BBM,23f03858-78a8-40df-9ff1-57fc137afe6a,22139
10524,1566221210450,BBM,d12502dd-d593-4d0d-a07f-53b78e06d9cc,0


In [493]:
policies[policies.uuid.isnull()]

,created_at,account_ref,uuid,price


In [494]:
mapping

,policy_id,pet_id
0,e8f83f3e-6f12-4b2b-9ee0-06a285afa04b,dcee9cc0-7987-40cb-a428-5bde29a1f057
1,7163b853-2cdc-4a9a-bac3-59c9c8d7e54c,8f672bf7-d4a2-4d1e-a3d2-a731c37dc9f2
2,f2a614fb-241f-42b8-a5c8-1f447fe46456,82d8fb94-0934-4b46-a189-a06708d9b25d
3,6718260e-b401-4e9d-aa91-269ef30e01e1,da9ced12-0dd9-4ab7-89e3-5e894a0cd334
4,7cf0888b-f699-4b7d-aac4-f0277ca72852,6c3a56d9-3a74-4b9f-b6fd-e52019fbb411
...,...,...
13212,db88159e-695e-4b1f-ba2d-6ac3d4a9e462,85ef0047-296b-49b5-af50-898e94adc24e
13213,886a3469-9a62-4646-bc19-41f59188e32d,f5bb306f-2e20-4e3c-87c6-c74d464cacc4
13214,23f03858-78a8-40df-9ff1-57fc137afe6a,b6521f93-b163-4f59-ab4b-34da1805b45f
13215,d12502dd-d593-4d0d-a07f-53b78e06d9cc,e9ecc594-b54b-47de-8630-3a53d6db2283


As expected, UUIDs are assigned for all 10526 policies, and all 13217 pets. Each pet is covered by exactly one policy.

We now apply some transformations to get our data into an appropriate format. I have chosen to create two tables linked by a foreign key, imitating a relational database.

In [495]:
policies['country'] = policies.account_ref.map(lambda x: 'UK' if x == 'BBM' else 'SWEDEN')

policies.set_index('uuid', inplace=True)
pets.set_index('uuid', inplace=True)
mapping.set_index('pet_id', inplace=True)

pets = pets.join(mapping)

In [496]:
csv_folder = f'{os.getcwd()}/data/csv'

policies.to_csv(f'{csv_folder}/policies.csv', columns=['country', 'created_at', 'price'])
pets.to_csv(f'{csv_folder}/pets.csv')

## 2. Reporting

Let's read our CSVs back in and begin computing metrics.

In [497]:
policies = pd.read_csv(f'{csv_folder}/policies.csv', index_col='uuid')
pets = pd.read_csv(f'{csv_folder}/pets.csv', index_col='uuid')

policies['purchase_date'] = policies.created_at.map(lambda x: datetime.fromtimestamp(x / 1000))
policies['month'] = policies.purchase_date.map(lambda x: x.month)

In [498]:
policies

,country,created_at,price,purchase_date,month
uuid,,,,,
e8f83f3e-6f12-4b2b-9ee0-06a285afa04b,UK,1564327896418,33543,2019-07-28 16:31:36.418,7
7163b853-2cdc-4a9a-bac3-59c9c8d7e54c,UK,1561825294796,43565,2019-06-29 17:21:34.796,6
f2a614fb-241f-42b8-a5c8-1f447fe46456,UK,1561050657706,25221,2019-06-20 18:10:57.706,6
6718260e-b401-4e9d-aa91-269ef30e01e1,UK,1553939844000,49315,2019-03-30 09:57:24.000,3
7cf0888b-f699-4b7d-aac4-f0277ca72852,UK,1569590695023,59144,2019-09-27 14:24:55.023,9
...,...,...,...,...,...
db88159e-695e-4b1f-ba2d-6ac3d4a9e462,UK,1563696195609,0,2019-07-21 09:03:15.609,7
886a3469-9a62-4646-bc19-41f59188e32d,UK,1556959333710,36807,2019-05-04 09:42:13.710,5
23f03858-78a8-40df-9ff1-57fc137afe6a,UK,1555184687260,22139,2019-04-13 20:44:47.260,4


In [499]:
pets

,breed,species,policy_id
uuid,,,
dcee9cc0-7987-40cb-a428-5bde29a1f057,PAPILLON,DOG,e8f83f3e-6f12-4b2b-9ee0-06a285afa04b
8f672bf7-d4a2-4d1e-a3d2-a731c37dc9f2,GERMAN_SHEPHERD,DOG,7163b853-2cdc-4a9a-bac3-59c9c8d7e54c
82d8fb94-0934-4b46-a189-a06708d9b25d,CAVAPOO,DOG,f2a614fb-241f-42b8-a5c8-1f447fe46456
da9ced12-0dd9-4ab7-89e3-5e894a0cd334,MALAMUTE_HUSKY,DOG,6718260e-b401-4e9d-aa91-269ef30e01e1
6c3a56d9-3a74-4b9f-b6fd-e52019fbb411,LARGE_MONGREL_(MORE_THAN_20KG),DOG,7cf0888b-f699-4b7d-aac4-f0277ca72852
...,...,...,...
85ef0047-296b-49b5-af50-898e94adc24e,COCKERPOO,DOG,db88159e-695e-4b1f-ba2d-6ac3d4a9e462
f5bb306f-2e20-4e3c-87c6-c74d464cacc4,LABRADOR_RETRIEVER,DOG,886a3469-9a62-4646-bc19-41f59188e32d
b6521f93-b163-4f59-ab4b-34da1805b45f,FRENCH_BULLDOG,DOG,23f03858-78a8-40df-9ff1-57fc137afe6a


We also need to read in the sales targets spreadsheet. I've assumed that the finance team are able to provide me with a simplified CSV file, which, for this exercise, I've put together manually. Let's take a look.

In [500]:
targets = pd.read_csv(f'{csv_folder}/targets.csv', index_col=['country', 'month'])
targets.value = targets.value.astype(float)

In [501]:
targets

quantity     value
country month                    
SWEDEN  7             5     682.0
        8            10     938.0
        9            15    1098.0
        10           25    1956.0
UK      2           600  206467.0
        3           750  264898.0
        4           893  271020.0
        5          1024  360561.0
        6          1160  392144.0
        7          1220  377063.0
        8          1315  408039.0
        9          1500  477178.0
        10         1650  575742.0

### Compare sales value by month against finance targets. Split by country.

We need to aggregrate our policy prices by month and country, remembering to convert Swedish Krona to Pound Sterling.

In [502]:
# current exchange rate
sek_to_gbp = 0.08

policies.price = policies.price / 100.0
policies.loc[policies.country == 'SWEDEN', 'price'] *= sek_to_gbp

In [503]:
performance = policies.groupby(['country', 'month']).price.agg(['count', 'sum'])
performance.columns = ['quantity', 'value']

In [504]:
performance

quantity      value
country month                     
SWEDEN  7             3    1098.96
        8             3    1331.52
        9             5    1514.72
        10           30   12433.60
UK      2           796  257941.20
        3           860  289544.45
        4           919  294995.52
        5           905  310637.02
        6           882  290746.70
        7          1201  417441.99
        8          1475  497295.06
        9          1645  540255.80
        10         1802  611898.18

In [505]:
comparison = targets.join(performance, lsuffix='_target', rsuffix='_actual')
comparison['quantity_difference'] = comparison.quantity_actual - comparison.quantity_target
comparison['quantity_percent'] = comparison.quantity_difference / comparison.quantity_target * 100
comparison.quantity_percent = comparison.quantity_percent.map(lambda x: f'{round(x, 1)}%')
comparison['value_difference'] = comparison.value_actual - comparison.value_target
comparison['value_percent'] = comparison.value_difference / comparison.value_target * 100
comparison.value_percent = comparison.value_percent.map(lambda x: f'{round(x, 1)}%')

comparison = comparison[[
    'quantity_target', 'quantity_actual', 'quantity_difference', 'quantity_percent',
    'value_target', 'value_actual', 'value_difference', 'value_percent'
]]

In [506]:
comparison

quantity_target  quantity_actual  quantity_difference  \
country month                                                          
SWEDEN  7                    5                3                   -2   
        8                   10                3                   -7   
        9                   15                5                  -10   
        10                  25               30                    5   
UK      2                  600              796                  196   
        3                  750              860                  110   
        4                  893              919                   26   
        5                 1024              905                 -119   
        6                 1160              882                 -278   
        7                 1220             1201                  -19   
        8                 1315             1475                  160   
        9                 1500             1645                  145   
        10                1650             1802                  152   

              quantity_percent  value_target  value_actual  value_difference  \
country month                                                                  
SWEDEN  7               -40.0%         682.0       1098.96            416.96   
        8               -70.0%         938.0       1331.52            393.52   
        9               -66.7%        1098.0       1514.72            416.72   
        10               20.0%        1956.0      12433.60          10477.60   
UK      2                32.7%      206467.0     257941.20          51474.20   
        3                14.7%      264898.0     289544.45          24646.45   
        4                 2.9%      271020.0     294995.52          23975.52   
        5               -11.6%      360561.0     310637.02         -49923.98   
        6               -24.0%      392144.0     290746.70        -101397.30   
        7                -1.6%      377063.0     417441.99          40378.99   
        8                12.2%      408039.0     497295.06          89256.06   
        9                 9.7%      477178.0     540255.80          63077.80   
        10                9.2%      575742.0     611898.18          36156.18   

              value_percent  
country month                
SWEDEN  7             61.1%  
        8             42.0%  
        9             38.0%  
        10           535.7%  
UK      2             24.9%  
        3              9.3%  
        4              8.8%  
        5            -13.8%  
        6            -25.9%  
        7             10.7%  
        8             21.9%  
        9             13.2%  
        10             6.3%

In the UK, we see strong sales performance in February, with the sales value target exceeded by about 25%. Sales declined from there, underperforming by about the same amount in June. By October, however, sales performance had recovered to about +6%.

In Sweden, value targets were significantly exceeded, despite selling fewer policies than expected. Clearly the average order value is higher than expected in the Swedish market. In October, sales quantity catches up and exceeds the target, and we also see an explosion in sales value

### Were UK sales above or below target in July? What was the difference?

The UK sales target was narrowly missed in July: 1201 sales against a target of 1220, a shortfall of 19 (-1.6%). However, the total sales value of £417,441.99 exceeded the £377,063 target (by £40,378.99, or 10.7%). Overall, a dramatic comeback from the June slump.

### What is the most common cat breed overall?

In [507]:
pets[pets.species == 'CAT'].groupby('breed').count().iloc[:, 0].sort_values(ascending=False)

breed
MOGGY                      1428
MOGGIE                      567
RAGDOLL                     112
DOMESTIC_SHORTHAIR          101
BRITISH_SHORTHAIR            83
                           ... 
FERAL                         1
CHINCHILLA_LONGHAIR           1
BURMESE_TORTOISESHELL         1
BURMESE_RED                   1
GREY_AND_WHITE_LONGHAIR       1
Name: species, Length: 115, dtype: int64

The Domestic Shorthair, or Moggy, is by far the most common cat breed in the dataset. 

If, however, we want to look at pedigree cats only, on first inspection we might conclude that the most common cat breed is the Ragdoll. However, closer inspection reveals that the British Shorthair, when all colour variants are accounted for, is actually the most common.

### What was the most common dog breed sold in UK in May and how many of that breed where there?

In [508]:
joined = pets.join(policies, on='policy_id')

In [509]:
joined

,breed,species,policy_id,country,created_at,price,purchase_date,month
uuid,,,,,,,,
dcee9cc0-7987-40cb-a428-5bde29a1f057,PAPILLON,DOG,e8f83f3e-6f12-4b2b-9ee0-06a285afa04b,UK,1564327896418,335.43,2019-07-28 16:31:36.418,7
8f672bf7-d4a2-4d1e-a3d2-a731c37dc9f2,GERMAN_SHEPHERD,DOG,7163b853-2cdc-4a9a-bac3-59c9c8d7e54c,UK,1561825294796,435.65,2019-06-29 17:21:34.796,6
82d8fb94-0934-4b46-a189-a06708d9b25d,CAVAPOO,DOG,f2a614fb-241f-42b8-a5c8-1f447fe46456,UK,1561050657706,252.21,2019-06-20 18:10:57.706,6
da9ced12-0dd9-4ab7-89e3-5e894a0cd334,MALAMUTE_HUSKY,DOG,6718260e-b401-4e9d-aa91-269ef30e01e1,UK,1553939844000,493.15,2019-03-30 09:57:24.000,3
6c3a56d9-3a74-4b9f-b6fd-e52019fbb411,LARGE_MONGREL_(MORE_THAN_20KG),DOG,7cf0888b-f699-4b7d-aac4-f0277ca72852,UK,1569590695023,591.44,2019-09-27 14:24:55.023,9
...,...,...,...,...,...,...,...,...
85ef0047-296b-49b5-af50-898e94adc24e,COCKERPOO,DOG,db88159e-695e-4b1f-ba2d-6ac3d4a9e462,UK,1563696195609,0.00,2019-07-21 09:03:15.609,7
f5bb306f-2e20-4e3c-87c6-c74d464cacc4,LABRADOR_RETRIEVER,DOG,886a3469-9a62-4646-bc19-41f59188e32d,UK,1556959333710,368.07,2019-05-04 09:42:13.710,5
b6521f93-b163-4f59-ab4b-34da1805b45f,FRENCH_BULLDOG,DOG,23f03858-78a8-40df-9ff1-57fc137afe6a,UK,1555184687260,221.39,2019-04-13 20:44:47.260,4


In [510]:
uk_may_dogs = joined[(joined.species == 'DOG') & (joined.country == 'UK') & (joined.month == 5)]

uk_may_dogs.groupby('breed').count().iloc[:, 0].sort_values(ascending=False)

breed
FRENCH_BULLDOG                75
LABRADOR_RETRIEVER            71
COCKER_SPANIEL                53
COCKERPOO                     46
MEDIUM_MONGREL_(10KG-20KG)    43
                              ..
LABRADOODLE_(MINIATURE)        1
LEONBERGER                     1
MALTI-POO                      1
MASTIFF                        1
AIREDALE_TERRIER               1
Name: species, Length: 139, dtype: int64

With 84 policies purchased, Labrador was the most commonly insured dog breed in the UK in May (again, excluding mixed breed dogs and including all colour variants).

### Sweden average order value in October was expected to be around 4000 Krona. Why was it noticeably different?

In [511]:
# calculate average order values in Sweden by month
(comparison.value_actual / comparison.quantity_actual).SWEDEN / sek_to_gbp

month
7     4579.000000
8     5548.000000
9     3786.800000
10    5180.666667
dtype: float64

Indeed, October surpassed expectations, with an average order value of about 5181 Krona. Inspecting the 30 orders in October revelas a particularly expensive one.

In [512]:
sweden_oct_sales = policies[(policies.month == 10) & (policies.country == 'SWEDEN')].sort_values('price', ascending=False)
sweden_oct_sales.iloc[:5, :]

,country,created_at,price,purchase_date,month
uuid,,,,,
63d5f678-3188-4636-9af7-4c03f8510163,SWEDEN,1570110097907,3031.76,2019-10-03 14:41:37.907,10
f0bc79ef-36dc-4b9d-bcf8-f4bf74c45507,SWEDEN,1572267544992,633.20,2019-10-28 12:59:04.992,10
ae6c8820-40ec-4440-bac7-485f8537c626,SWEDEN,1571405287909,580.16,2019-10-18 14:28:07.909,10
3ffbf60f-bfbb-445d-8012-e550c2dfe168,SWEDEN,1571767944013,540.16,2019-10-22 19:12:24.013,10
434fde3c-375a-46f1-b5ce-dbc96a562260,SWEDEN,1571323252929,520.64,2019-10-17 15:40:52.929,10


Excluding the anomaly, we get a much more reasonable average of 4053 Krona. Further inspection reveals that a customer took out a single policy covering 6 dogs and 6 cats, hence the high price tag.

In [513]:
sweden_oct_sales.drop('63d5f678-3188-4636-9af7-4c03f8510163').price.mean() / sek_to_gbp

4052.5172413793116

In [514]:
joined[joined.policy_id == '63d5f678-3188-4636-9af7-4c03f8510163']

,breed,species,policy_id,country,created_at,price,purchase_date,month
uuid,,,,,,,,
d474abdf-44f1-430c-a7c1-082fef190995,"{'name': 'Tysk schäferhund', 'objectID': 'ALSA...",DOG,63d5f678-3188-4636-9af7-4c03f8510163,SWEDEN,1570110097907,3031.76,2019-10-03 14:41:37.907,10
705a5875-aac8-40e7-b87f-713501a6d003,"{'name': 'Pomeranian', 'objectID': 'POMERANIAN'}",DOG,63d5f678-3188-4636-9af7-4c03f8510163,SWEDEN,1570110097907,3031.76,2019-10-03 14:41:37.907,10
f48676cb-5c34-4aff-86f8-0abdd78372f1,"{'name': 'Pudel toy', 'objectID': 'TOY_POODLE'}",DOG,63d5f678-3188-4636-9af7-4c03f8510163,SWEDEN,1570110097907,3031.76,2019-10-03 14:41:37.907,10
d052dc82-b141-4b49-a452-65e28ba0d184,"{'name': 'Svensk lapphund', 'objectID': 'SWEDI...",DOG,63d5f678-3188-4636-9af7-4c03f8510163,SWEDEN,1570110097907,3031.76,2019-10-03 14:41:37.907,10
46d14fd5-2483-48bf-9af1-8c22a8db721d,"{'name': 'Leonberger', 'objectID': 'LEONBERGER'}",DOG,63d5f678-3188-4636-9af7-4c03f8510163,SWEDEN,1570110097907,3031.76,2019-10-03 14:41:37.907,10
a5dc2ff2-7128-47e5-ab2b-f6b5df2a6379,"{'name': 'Sibirisk katt', 'objectID': 'SIBERIAN'}",CAT,63d5f678-3188-4636-9af7-4c03f8510163,SWEDEN,1570110097907,3031.76,2019-10-03 14:41:37.907,10
486b7e63-c68b-4b3d-a964-20cac578e20a,"{'name': 'Sibirisk katt', 'objectID': 'SIBERIAN'}",CAT,63d5f678-3188-4636-9af7-4c03f8510163,SWEDEN,1570110097907,3031.76,2019-10-03 14:41:37.907,10
ba38c6c6-af03-465e-90c7-4fea259a688a,"{'name': 'blandras', 'objectID': 'DOMESTIC_SHO...",CAT,63d5f678-3188-4636-9af7-4c03f8510163,SWEDEN,1570110097907,3031.76,2019-10-03 14:41:37.907,10
77980e8a-7629-4b0e-8a39-87a94ca2e131,"{'name': 'blandras', 'objectID': 'DOMESTIC_SHO...",CAT,63d5f678-3188-4636-9af7-4c03f8510163,SWEDEN,1570110097907,3031.76,2019-10-03 14:41:37.907,10
